In [1]:
%cd ..

/Users/shivamkaushik/Code/ik-agent


In [ ]:
from typing import Any, Dict, Optional, Union, Literal
from pydantic import (
    BaseModel,
    Field,
    field_validator,
)
from src.modules.ik_transforms.types import (
    NumberOrExpression,
    FONT_FAMILY,
    FONT_FILE_REGEX,
    PaddingValue,
    AlphaLevel,
    FlipMode,
)

In [3]:
from src.modules.ik_transforms.transforms.text_overlay import TextOverlay

# from src.tools.assets.list_assets import list_assets
# class TextOverlay(BaseModel):
#     text: str
#     lx: Optional[NumberOrExpression] = None
#     ly: Optional[NumberOrExpression] = None
#     width: Optional[NumberOrExpression] = None
#     font_size: Optional[NumberOrExpression] = None
#     font_family: Optional[str] = None
#     color: Optional[str] = None
#     inner_alignment: Optional[Literal["left", "center", "right"]] = None
#     padding: Optional[PaddingValue] = None
#     alpha: Optional[AlphaLevel] = None
#     typography: Optional[Literal[
#         'b',
#         'i',
#         'strikethrough',
#         'b_i',
#         'b_strikethrough',
#         'i_strikethrough',
#         'b_i_strikethrough',
#     ]] = None
#     background: Optional[str] = None
#     corner_radius: Optional[NumberOrExpression] = None
#     rotation: Optional[NumberOrExpression] = None
#     flip: Optional[FlipMode] = None
#     line_height: Optional[NumberOrExpression] = None


#     @field_validator("font_family")
#     @classmethod
#     def validate_font_family(cls, v: Optional[str]) -> Optional[str]:
#         if v is None:
#             return v

#         # Case 1: Standard ImageKit font
#         if v in FONT_FAMILY:
#             return v

#         # Case 2: Custom font file path
#         if FONT_FILE_REGEX.fullmatch(v):
#             return v

#         raise ValueError(
#             f"Invalid font_family '{v}'. "
#             "Must be a supported ImageKit font name or "
#             "a valid custom font file path (e.g. fonts@@MyFont.ttf)."
#         )

#     @staticmethod
#     async def ensure_font_exists(font_path: str):
#         files = await list_assets(search_query=f'name="{font_path}"', limit=1)
#         if not files:
#             raise ValueError(f"Font file '{font_path}' not found")


#     async def to_overlay_dict(self) -> Dict[str, Any]:
#         """
#         Convert the overlay model into the nested dict structure expected by
#         the ImageKit URL builder.

#         Uses `model_dump()` to ensure all NumberOrExpression fields are
#         serialized correctly (e.g. negative numbers -> 'N{abs(x)}').
#         """
#         dumped = self.model_dump(exclude_none=True)

#         overlay: Dict[str, Any] = {
#             "type": "text",
#             "text": dumped["text"],
#         }

#         # -------------------------------------------------
#         # POSITION
#         # -------------------------------------------------
#         position: Dict[str, Any] = {}

#         if "lx" in dumped:
#             position["x"] = dumped["lx"]

#         if "ly" in dumped:
#             position["y"] = dumped["ly"]

#         if position:
#             overlay["position"] = position

#         # -------------------------------------------------
#         # TRANSFORMATIONS
#         # -------------------------------------------------
#         transform: Dict[str, Any] = {}

#         if "width" in dumped:
#             transform["width"] = dumped["width"]

#         if "font_size" in dumped:
#             transform["font_size"] = dumped["font_size"]

#         if "font_family" in dumped:
#             if dumped["font_family"] not in FONT_FAMILY:
#                 await self.ensure_font_exists(dumped["font_family"])
#             transform["font_family"] = dumped["font_family"]

#         if "color" in dumped:
#             transform["font_color"] = dumped["color"]

#         if "typography" in dumped:
#             overlay["typography"] = dumped["typography"]

#         if "inner_alignment" in dumped:
#             transform["inner_alignment"] = dumped["inner_alignment"]

#         if "alpha" in dumped:
#             transform["alpha"] = dumped["alpha"]

#         if "padding" in dumped:
#             transform["padding"] = dumped["padding"]

#         if "background" in dumped:
#             transform["background"] = dumped["background"]

#         if "corner_radius" in dumped:
#             transform["radius"] = dumped["corner_radius"]

#         if "rotation" in dumped:
#             transform["rotation"] = dumped["rotation"]

#         if "flip" in dumped:
#             transform["flip"] = dumped["flip"]

#         if "line_height" in dumped:
#             transform["line_height"] = dumped["line_height"]

#         overlay["transformation"] = [transform]

#         return {"overlay": overlay}


Deprecation warning: AnalyticsRulesV1 is deprecated on v30+. Use client.analytics instead.


In [ ]:
import os
import logging
from imagekitio import ImageKit
from IPython.display import Image, display


client = ImageKit(
    private_key=os.environ.get("IMAGEKIT_PRIVATE_KEY"),
)


def build_url(
    url_endpoint: str,
    src: str,
    transformation=None,
) -> str:
    if isinstance(transformation, dict):
        transformation = [transformation]
    url = client.helper.build_url(
        url_endpoint=url_endpoint,
        src=src,
        transformation=transformation,
    )
    # print(url)
    display(Image(url=url))
    return url


In [5]:
params = {"text": "Overlay made easy", "font_size": 45, "lx": 35}

transforms = await TextOverlay(**params).to_overlay_dict()
print(transforms)
build_url(
    url_endpoint="https://ik.imagekit.io/demo",
    src="medium_cafe_B1iTdD0C.jpg",
    transformation=transforms,
)

layer_mode_obj: None
{'overlay': {'type': 'text', 'text': 'Overlay made easy', 'position': {'x': '35'}, 'transformation': [{'font_size': '45'}]}}


'https://ik.imagekit.io/demo/medium_cafe_B1iTdD0C.jpg?tr=l-text,i-Overlay%20made%20easy,lx-35,fs-45,l-end'

In [6]:
params = {"text": "Overlay made easy", "font_size": 45, "lx": -35}

transforms = await TextOverlay(**params).to_overlay_dict()
print(transforms)
build_url(
    url_endpoint="https://ik.imagekit.io/demo",
    src="medium_cafe_B1iTdD0C.jpg",
    transformation=transforms,
)

layer_mode_obj: None
{'overlay': {'type': 'text', 'text': 'Overlay made easy', 'position': {'x': 'N35'}, 'transformation': [{'font_size': '45'}]}}


'https://ik.imagekit.io/demo/medium_cafe_B1iTdD0C.jpg?tr=l-text,i-Overlay%20made%20easy,lx-N35,fs-45,l-end'

In [7]:
params = {
    "text": "Overlay made easy",
    "font_size": 45,
    "lx": "bw_mul_0.1",
    "ly": "bh_mul_0.1",
}

transforms = await TextOverlay(**params).to_overlay_dict()
print(transforms)
build_url(
    url_endpoint="https://ik.imagekit.io/demo",
    src="medium_cafe_B1iTdD0C.jpg",
    transformation=transforms,
)

layer_mode_obj: None
{'overlay': {'type': 'text', 'text': 'Overlay made easy', 'position': {'x': 'bw_mul_0.1', 'y': 'bh_mul_0.1'}, 'transformation': [{'font_size': '45'}]}}


'https://ik.imagekit.io/demo/medium_cafe_B1iTdD0C.jpg?tr=l-text,i-Overlay%20made%20easy,lx-bw_mul_0.1,ly-bh_mul_0.1,fs-45,l-end'

In [8]:
params = {
    "text": "Overlay made easy",
    "font_size": 45,
    # "lx": 'bw_mul_0.1',
    "ly": "bh_mul_0.8",
}

transforms = await TextOverlay(**params).to_overlay_dict()
print(transforms)
build_url(
    url_endpoint="https://ik.imagekit.io/demo",
    src="medium_cafe_B1iTdD0C.jpg",
    transformation=transforms,
)

layer_mode_obj: None
{'overlay': {'type': 'text', 'text': 'Overlay made easy', 'position': {'y': 'bh_mul_0.8'}, 'transformation': [{'font_size': '45'}]}}


'https://ik.imagekit.io/demo/medium_cafe_B1iTdD0C.jpg?tr=l-text,i-Overlay%20made%20easy,ly-bh_mul_0.8,fs-45,l-end'

In [9]:
params1 = {
    "text": "overlay made easy1",
    "font_size": 45,
    "ly": "bh_mul_0.2",
    "width": "bw_mul_0.5",
    "background": "yellow",
}
transforms1 = await TextOverlay(**params1).to_overlay_dict()

params2 = {
    "text": "overlay made easy2",
    "font_size": 45,
    "ly": "bh_mul_0.2",
    "lx": "bw_mul_0.5",
    "width": "bw_mul_0.5",
    "background": "orange",
}
transforms2 = await TextOverlay(**params2).to_overlay_dict()
transformations = [transforms1, transforms2]

print(transforms)
build_url(
    url_endpoint="https://ik.imagekit.io/demo",
    src="medium_cafe_B1iTdD0C.jpg",
    transformation=transformations,
)

layer_mode_obj: None
layer_mode_obj: None
{'overlay': {'type': 'text', 'text': 'Overlay made easy', 'position': {'y': 'bh_mul_0.8'}, 'transformation': [{'font_size': '45'}]}}


'https://ik.imagekit.io/demo/medium_cafe_B1iTdD0C.jpg?tr=l-text,i-overlay%20made%20easy1,ly-bh_mul_0.2,w-bw_mul_0.5,fs-45,bg-yellow,l-end:l-text,i-overlay%20made%20easy2,lx-bw_mul_0.5,ly-bh_mul_0.2,w-bw_mul_0.5,fs-45,bg-orange,l-end'

In [10]:
params = {
    "text": "Overlay made easy",
    "font_size": 45,
    "padding": 10,
    "background": "white",
}

transforms = await TextOverlay(**params).to_overlay_dict()
print(transforms)
build_url(
    url_endpoint="https://ik.imagekit.io/demo",
    src="medium_cafe_B1iTdD0C.jpg",
    transformation=transforms,
)

layer_mode_obj: None
{'overlay': {'type': 'text', 'text': 'Overlay made easy', 'transformation': [{'font_size': '45', 'padding': '10', 'background': 'white'}]}}


'https://ik.imagekit.io/demo/medium_cafe_B1iTdD0C.jpg?tr=l-text,i-Overlay%20made%20easy,fs-45,pa-10,bg-white,l-end'

In [11]:
params = {
    "text": "Overlay made easy",
    "font_size": 45,
    "padding": 40,
    "background": "white",
    "width": 300,
}

transforms = await TextOverlay(**params).to_overlay_dict()
print(transforms)
build_url(
    url_endpoint="https://ik.imagekit.io/demo",
    src="medium_cafe_B1iTdD0C.jpg",
    transformation=transforms,
)

layer_mode_obj: None
{'overlay': {'type': 'text', 'text': 'Overlay made easy', 'transformation': [{'width': '300', 'font_size': '45', 'padding': '40', 'background': 'white'}]}}


'https://ik.imagekit.io/demo/medium_cafe_B1iTdD0C.jpg?tr=l-text,i-Overlay%20made%20easy,w-300,fs-45,pa-40,bg-white,l-end'

In [12]:
params = {
    "text": "Overlay made easy",
    "font_size": 45,
    "padding": 40,
    "background": "white",
    "width": 300,
    "alpha": 9,
}

transforms = await TextOverlay(**params).to_overlay_dict()
print(transforms)
build_url(
    url_endpoint="https://ik.imagekit.io/demo",
    src="medium_cafe_B1iTdD0C.jpg",
    transformation=transforms,
)

layer_mode_obj: None
{'overlay': {'type': 'text', 'text': 'Overlay made easy', 'transformation': [{'width': '300', 'font_size': '45', 'alpha': '9', 'padding': '40', 'background': 'white'}]}}


'https://ik.imagekit.io/demo/medium_cafe_B1iTdD0C.jpg?tr=l-text,i-Overlay%20made%20easy,w-300,fs-45,al-9,pa-40,bg-white,l-end'

In [13]:
params = {
    "text": "Overlay made easy",
    "width": 150,
    "font_size": 45,
    # "padding": 40,
    "background": "white",
    "line_height": 20,
}

transforms = await TextOverlay(**params).to_overlay_dict()
print(transforms)
build_url(
    url_endpoint="https://ik.imagekit.io/demo",
    src="medium_cafe_B1iTdD0C.jpg",
    transformation=transforms,
)

layer_mode_obj: None
{'overlay': {'type': 'text', 'text': 'Overlay made easy', 'transformation': [{'width': '150', 'font_size': '45', 'background': 'white', 'line_height': '20'}]}}


'https://ik.imagekit.io/demo/medium_cafe_B1iTdD0C.jpg?tr=l-text,i-Overlay%20made%20easy,w-150,fs-45,bg-white,lh-20,l-end'

Top padding is 25

Right padding is 50

Bottom padding is 75

Left padding is 100

In [14]:
params = {
    "text": "Overlay made easy",
    "font_size": 45,
    "padding": "25_50_75_100",
    "background": "white",
}

transforms = await TextOverlay(**params).to_overlay_dict()
print(transforms)
build_url(
    url_endpoint="https://ik.imagekit.io/demo",
    src="medium_cafe_B1iTdD0C.jpg",
    transformation=transforms,
)

layer_mode_obj: None
{'overlay': {'type': 'text', 'text': 'Overlay made easy', 'transformation': [{'font_size': '45', 'padding': '25_50_75_100', 'background': 'white'}]}}


'https://ik.imagekit.io/demo/medium_cafe_B1iTdD0C.jpg?tr=l-text,i-Overlay%20made%20easy,fs-45,pa-25_50_75_100,bg-white,l-end'

In [15]:
params = {
    "text": "Overlay made easy",
    "font_size": 45,
    "padding": "25_75_60",
    "background": "white",
}

transforms = await TextOverlay(**params).to_overlay_dict()
print(transforms)
build_url(
    url_endpoint="https://ik.imagekit.io/demo",
    src="medium_cafe_B1iTdD0C.jpg",
    transformation=transforms,
)

layer_mode_obj: None
{'overlay': {'type': 'text', 'text': 'Overlay made easy', 'transformation': [{'font_size': '45', 'padding': '25_75_60', 'background': 'white'}]}}


'https://ik.imagekit.io/demo/medium_cafe_B1iTdD0C.jpg?tr=l-text,i-Overlay%20made%20easy,fs-45,pa-25_75_60,bg-white,l-end'

In [16]:
params = {
    "text": "Overlay made easy",
    "font_size": 45,
    "padding": "25_75",
    "background": "white",
}

transforms = await TextOverlay(**params).to_overlay_dict()
print(transforms)
build_url(
    url_endpoint="https://ik.imagekit.io/demo",
    src="medium_cafe_B1iTdD0C.jpg",
    transformation=transforms,
)

layer_mode_obj: None
{'overlay': {'type': 'text', 'text': 'Overlay made easy', 'transformation': [{'font_size': '45', 'padding': '25_75', 'background': 'white'}]}}


'https://ik.imagekit.io/demo/medium_cafe_B1iTdD0C.jpg?tr=l-text,i-Overlay%20made%20easy,fs-45,pa-25_75,bg-white,l-end'

In [17]:
params = {
    "text": "Overlay made easy",
    "font_size": 45,
    "padding": 25,
    "background": "white",
    "font_family": "Arial.ttf",
    # "font_family": '/fonts/v1.2/Poppins-Regular_Q15GrYWmL.ttf'
}
try:
    transforms = await TextOverlay(**params).to_overlay_dict()

    print(transforms)
    build_url(
        url_endpoint="https://ik.imagekit.io/demo",
        src="medium_cafe_B1iTdD0C.jpg",
        transformation=transforms,
    )
except ValueError as e:
    print(f"Error: {e}")

Error: Font file 'Arial.ttf' not found


In [18]:
params = {
    "text": "Overlay made easy",
    "font_size": 45,
    "padding": 25,
    "background": "white",
    "font_family": "love_days.ttf",
    # "font_family": '/fonts/v1.2/Poppins-Regular_Q15GrYWmL.ttf'
}

transforms = await TextOverlay(**params).to_overlay_dict()
print(transforms)
build_url(
    url_endpoint="https://ik.imagekit.io/demo",
    src="medium_cafe_B1iTdD0C.jpg",
    transformation=transforms,
)

layer_mode_obj: None
{'overlay': {'type': 'text', 'text': 'Overlay made easy', 'transformation': [{'font_size': '45', 'font_family': 'love_days.ttf', 'padding': '25', 'background': 'white'}]}}


'https://ik.imagekit.io/demo/medium_cafe_B1iTdD0C.jpg?tr=l-text,i-Overlay%20made%20easy,fs-45,ff-love_days.ttf,pa-25,bg-white,l-end'

In [19]:
params = {
    "text": "Overlay made easy",
    "font_size": 45,
    "padding": 25,
    "background": "white",
    "font_family": "Monoton",
    # "font_family": '/fonts/v1.2/Poppins-Regular_Q15GrYWmL.ttf'
}

transforms = await TextOverlay(**params).to_overlay_dict()
print(transforms)
build_url(
    url_endpoint="https://ik.imagekit.io/demo",
    src="medium_cafe_B1iTdD0C.jpg",
    transformation=transforms,
)

layer_mode_obj: None
{'overlay': {'type': 'text', 'text': 'Overlay made easy', 'transformation': [{'font_size': '45', 'font_family': 'Monoton', 'padding': '25', 'background': 'white'}]}}


'https://ik.imagekit.io/demo/medium_cafe_B1iTdD0C.jpg?tr=l-text,i-Overlay%20made%20easy,fs-45,ff-Monoton,pa-25,bg-white,l-end'

In [20]:
params = {
    "text": "Overlay made easy",
    "font_size": 45,
    "flip": "h_v",
    "background": "white",
}

transforms = await TextOverlay(**params).to_overlay_dict()
print(transforms)
build_url(
    url_endpoint="https://ik.imagekit.io/demo",
    src="medium_cafe_B1iTdD0C.jpg",
    transformation=transforms,
)

layer_mode_obj: None
{'overlay': {'type': 'text', 'text': 'Overlay made easy', 'transformation': [{'font_size': '45', 'background': 'white', 'flip': 'h_v'}]}}


'https://ik.imagekit.io/demo/medium_cafe_B1iTdD0C.jpg?tr=l-text,i-Overlay%20made%20easy,fs-45,bg-white,fl-h_v,l-end'

In [24]:
params = {
    "text": "Overlay made easy",
    "font_size": 60,
    "layer_mode": "multiply",
    "color": "red",
    "typography": "b",
}

transforms = await TextOverlay(**params).to_overlay_dict()
print(transforms)
build_url(
    url_endpoint="https://ik.imagekit.io/demo",
    src="medium_cafe_B1iTdD0C.jpg",
    transformation=transforms,
)

layer_mode_obj: mode='multiply'
{'overlay': {'type': 'text', 'text': 'Overlay made easy', 'typography': 'b', 'transformation': [{'font_size': '60', 'font_color': 'red', 'lm': 'multiply'}]}}


'https://ik.imagekit.io/demo/medium_cafe_B1iTdD0C.jpg?tr=l-text,i-Overlay%20made%20easy,fs-60,co-red,lm-multiply,l-end'

In [32]:
params = {
    "text": "Overlay made easy",
    "font_size": 60,
    "layer_mode": "cutout",
    "typography": "b",
}

transforms = await TextOverlay(**params).to_overlay_dict()
print(transforms)
build_url(
    url_endpoint="https://ik.imagekit.io/demo",
    src="medium_cafe_B1iTdD0C.jpg",
    transformation=[transforms, {"bg": "red"}],
)

layer_mode_obj: mode='cutout'
{'overlay': {'type': 'text', 'text': 'Overlay made easy', 'typography': 'b', 'transformation': [{'font_size': '60', 'lm': 'cutout'}]}}


'https://ik.imagekit.io/demo/medium_cafe_B1iTdD0C.jpg?tr=l-text,i-Overlay%20made%20easy,fs-60,lm-cutout,l-end:bg-red'

In [25]:
params = {
    "text": "Overlay made easy",
    "font_size": 60,
    "color": "red",
    "typography": "b",
}

transforms = await TextOverlay(**params).to_overlay_dict()
print(transforms)
build_url(
    url_endpoint="https://ik.imagekit.io/demo",
    src="medium_cafe_B1iTdD0C.jpg",
    transformation=transforms,
)

layer_mode_obj: None
{'overlay': {'type': 'text', 'text': 'Overlay made easy', 'typography': 'b', 'transformation': [{'font_size': '60', 'font_color': 'red'}]}}


'https://ik.imagekit.io/demo/medium_cafe_B1iTdD0C.jpg?tr=l-text,i-Overlay%20made%20easy,fs-60,co-red,l-end'

In [33]:
params = {
    "text": "Overlay made easy",
    "font_size": 60,
    "color": "red",
    "typography": "b",
    "layer_mode": "cutter",
}

transforms = await TextOverlay(**params).to_overlay_dict()
print(transforms)
build_url(
    url_endpoint="https://ik.imagekit.io/demo/",
    src="medium_cafe_B1iTdD0C.jpg",
    transformation=transforms,
)



layer_mode_obj: mode='cutter'
{'overlay': {'type': 'text', 'text': 'Overlay made easy', 'typography': 'b', 'transformation': [{'font_size': '60', 'font_color': 'red', 'lm': 'cutter'}]}}


'https://ik.imagekit.io/demo/medium_cafe_B1iTdD0C.jpg?tr=l-text,i-Overlay%20made%20easy,fs-60,co-red,lm-cutter,l-end'